In [8]:
import os
import logging
import argparse
import random
from tqdm import tqdm, trange
from torch.utils.data import DataLoader
import numpy as np
from shutil import copyfile
import torch
from src.KEMCE.knowledge_bert import SeqsTokenizer, EntityTokenizer, DescTokenizer, \
    KemceForPreTraining, BertAdam, BertConfig
from src.KEMCE.dataset import prepare_data, BERTDataset, collate_mlm

ModuleNotFoundError: No module named 'knowledge_bert'

In [2]:
pre_trained_path = './outputs/kemce/models/pre_trained_np'

In [5]:
weights_path = os.path.join(pre_trained_path, 'pytorch_model.bin_0')
state_dict = torch.load(weights_path)
model, _ = KemceForPreTraining.from_pretrained(pre_trained_path, state_dict)

parameters in inputs: 


In [6]:
model

KemceForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(6994, 100)
      (token_type_embeddings): Embedding(2, 100)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer_simple(
          (attention): BertAttention_simple(
            (self): BertSelfAttention(
              (query): Linear(in_features=100, out_features=100, bias=True)
              (key): Linear(in_features=100, out_features=100, bias=True)
              (value): Linear(in_features=100, out_features=100, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=100, out_features=100, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (intermediate): BertIntermed

In [4]:
data_path = './'
out_dir = data_path + 'outputs/kemce/data/raw/'
seqs_file = data_path + 'outputs/kemce/data/raw/mimic_pre_train.seqs'
dict_file = data_path + 'outputs/kemce/data/raw/mimic_pre_train_vocab.txt'

code2desc_file = data_path + 'outputs/kemce/KG/code2desc.pickle'
ent_vocab_file = data_path + 'outputs/kemce/KG/entity2id'

ent_embd_file = data_path + 'outputs/kemce/KG/embeddings/CCS_TransR_entity.npy'

config_json = data_path + 'src/KEMCE/kemce_config.json'

In [5]:
ent_embd = np.load(ent_embd_file)
ent_embd = torch.tensor(ent_embd)
# padding for special word "unknown" 
pad_embed = torch.zeros(1,ent_embd.shape[1])
ent_embd = torch.cat([pad_embed, ent_embd])
ent_embedding = nn.Embedding.from_pretrained(ent_embd, freeze=True)

In [6]:
seq_tokenizer = SeqsTokenizer(dict_file)
ent_tokenize = EntityTokenizer(ent_vocab_file)
desc_tokenize = DescTokenizer(code2desc_file)

In [7]:
pair_seqs, pair_seqs_mask, pair_seqs_mask_label = prepare_data(seq_tokenizer.ids_to_tokens, seqs_file, out_dir)

In [1]:
seq_tokenizer.vovab

NameError: name 'seq_tokenizer' is not defined

In [8]:
dataset = BERTDataset(pair_seqs, pair_seqs_mask, pair_seqs_mask_label, seq_tokenizer, ent_tokenize, desc_tokenize)

In [26]:
train_data_loader = DataLoader(dataset, batch_size=64, 
                               collate_fn=lambda batch: collate_mlm(batch, ent_embedding),
                               num_workers=0, shuffle=True)
dataiter = iter(train_data_loader)

In [27]:
data = dataiter.next()

In [28]:
batch = {k:t for k, t in data.items()}

In [29]:
batch

{'mlm_input': tensor([[   2,    4,  111,  ...,    0,    0,    0],
         [   2,   86,  134,  ...,    0,    0,    0],
         [   2,   67,  305,  ...,    0,    0,    0],
         ...,
         [   2, 2633,   14,  ...,    0,    0,    0],
         [   2,   25, 3864,  ..., 5467,  184,    3],
         [   2, 3242,  340,  ...,    0,    0,    0]]),
 'mlm_label': tensor([[  0, 193,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0, 105,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]]),
 'ent_input': tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [-0.1586, -0.1068,  0.2344,  ..., -0.0978,  0.0071, -0.0172],
          [ 0.0836, -0.1627,  0.0898,  ...,  0.1982, -0.0763, -0.0118],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.000

In [ ]:
# embedding entity ids
seq_input_tensor = data['mlm_input'][8].to(device)
token_type_tensor = data['token_type'][8].to(device)
ent_input_tensor = data['ent_input'][8].to(device)
desc_input_tensor = data['desc_input'][8].to(device)
ent_mask_tensor = data['ent_mask'][8].to(device)
input_mask_tensor = data['input_mask'][8].to(device)

In [11]:
config = BertConfig.from_json_file(config_json)

model = KemceForPreTraining(config)

In [12]:
model

KemceForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(6994, 100)
      (position_embeddings): Embedding(100, 100)
      (token_type_embeddings): Embedding(2, 100)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer_simple(
          (attention): BertAttention_simple(
            (self): BertSelfAttention(
              (query): Linear(in_features=100, out_features=100, bias=True)
              (key): Linear(in_features=100, out_features=100, bias=True)
              (value): Linear(in_features=100, out_features=100, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=100, out_features=100, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
 

In [13]:
param_optimizer = list(model.named_parameters())

In [14]:
param_optimizer

[('bert.embeddings.word_embeddings.weight',
  Parameter containing:
  tensor([[-0.0153,  0.0045, -0.0277,  ..., -0.0063,  0.0296, -0.0533],
          [ 0.0204, -0.0364, -0.0289,  ..., -0.0003,  0.0076,  0.0157],
          [-0.0014,  0.0256, -0.0118,  ...,  0.0198, -0.0027, -0.0391],
          ...,
          [-0.0100, -0.0169,  0.0068,  ...,  0.0068, -0.0092, -0.0024],
          [-0.0219,  0.0016,  0.0279,  ..., -0.0168, -0.0233, -0.0005],
          [-0.0140, -0.0010,  0.0059,  ...,  0.0064, -0.0048, -0.0327]],
         requires_grad=True)),
 ('bert.embeddings.position_embeddings.weight',
  Parameter containing:
  tensor([[-0.0114,  0.0104,  0.0035,  ...,  0.0004, -0.0063, -0.0060],
          [ 0.0051, -0.0092,  0.0149,  ..., -0.0108, -0.0085,  0.0202],
          [ 0.0321,  0.0033, -0.0153,  ..., -0.0312,  0.0266,  0.0186],
          ...,
          [-0.0105, -0.0004,  0.0018,  ..., -0.0113,  0.0447,  0.0023],
          [ 0.0175, -0.0074, -0.0040,  ..., -0.0098, -0.0153, -0.0176],
      

In [26]:
import torch
from torch import nn

from src.KEMCE.knowledge_bert import SeqsTokenizer, EntityTokenizer, DescTokenizer, \
    KemceForPreTraining
import numpy as np

data_path = './'
dict_file = data_path + 'outputs/kemce/data/raw/mimic_pre_train_vocab.txt'
code2desc_file = data_path + 'outputs/kemce/KG/code2desc.pickle'
ent_vocab_file = data_path + 'outputs/kemce/KG/entity2id'
ent_embd_file = data_path + 'outputs/kemce/KG/embeddings/CCS_TransR_entity.npy'

ent_embd = np.load(ent_embd_file)
ent_embd = torch.tensor(ent_embd)
# padding for special word "unknown"
pad_embed = torch.zeros(1,ent_embd.shape[1])
ent_embd = torch.cat([pad_embed, ent_embd])
ent_embedding = nn.Embedding.from_pretrained(ent_embd, freeze=True)

seq_tokenizer = SeqsTokenizer(dict_file)
ent_tokenize = EntityTokenizer(ent_vocab_file)
desc_tokenize = DescTokenizer(code2desc_file)

# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

# visit = '[CLS] D_41401 D_V4581 D_53081 D_496 D_30000 [SEP] D_4241 D_2720 D_V4581 D_4538 [SEP]'
# mask_input = '[CLS] D_41401 D_V4581 [MASK] D_496 D_30000 [SEP] D_4241 D_2720 D_V4581 D_4538 [SEP]'
# mask_label = '[PAD] [PAD] [PAD] D_53081 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'
# ent_input_str = '[UNK] D_41401 D_V4581 D_53081 D_496 D_30000 [UNK] D_4241 D_2720 D_V4581 D_4538 [UNK]'
# token_type = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
# ent_mask =   [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0]
# input_mask = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

# _, mask_input = seq_tokenizer.tokenize(mask_input)
# _, mask_label = seq_tokenizer.tokenize(mask_label)
# _, ent_input = ent_tokenize.tokenize(ent_input_str)
# _, desc_input = desc_tokenize.tokenize(ent_input_str)

# # embedding entity ids
# ent_input_tensor = torch.tensor([ent_input]).long()
# ent_input_embed = ent_embedding(ent_input_tensor).to(device)

# seq_input_tensor = torch.tensor([mask_input]).long().to(device)
# token_type_tensor = torch.tensor([token_type]).long().to(device)
# desc_input_tensor = torch.tensor([desc_input]).long().to(device)
# ent_mask_tensor = torch.tensor([ent_mask]).to(device)
# input_mask_tensor = torch.tensor([input_mask]).to(device)

# embedding entity ids
seq_input_tensor = torch.tensor([data['mlm_input'][8].numpy()]).long().to(device)
token_type_tensor = torch.tensor([data['token_type'][8].numpy()]).long().to(device)
ent_input_tensor = torch.tensor([data['ent_input'][8].numpy()]).to(device)
desc_input_tensor = torch.tensor([data['desc_input'][8].numpy()]).long().to(device)
ent_mask_tensor = torch.tensor([data['ent_mask'][8].numpy()]).to(device)
input_mask_tensor = torch.tensor([data['input_mask'][8].numpy()]).to(device)

masked_index = 3
mask_word = 'D_53081'

# training from pre-trained model
model, _ = KemceForPreTraining.from_pretrained('kemce_pre_trained')
model = model.to(device)
model.eval()


# Predict all tokens
with torch.no_grad():

    prediction, _ = model(seq_input_tensor, token_type_tensor, ent_input_tensor,
                     desc_input_tensor, ent_mask_tensor, input_mask_tensor)
    predicted_index = torch.argmax(prediction[0, masked_index]).item()

    predicted_token = seq_tokenizer.convert_ids_to_tokens([predicted_index])[0]

print(mask_word, predicted_token)

parameters in inputs: 
D_53081 [PAD]


In [30]:
model

KemceForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(6994, 100)
      (token_type_embeddings): Embedding(2, 100)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer_simple(
          (attention): BertAttention_simple(
            (self): BertSelfAttention(
              (query): Linear(in_features=100, out_features=100, bias=True)
              (key): Linear(in_features=100, out_features=100, bias=True)
              (value): Linear(in_features=100, out_features=100, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=100, out_features=100, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (intermediate): BertIntermed

In [34]:
model.bert.embeddings.word_embeddings.weight[79]

tensor([-2.3817e-01,  3.3153e+00,  9.9319e-02,  6.3417e+00,  6.9359e+00,
         2.9401e+00, -8.3811e-01, -2.0961e+00, -7.9027e-01, -2.6988e+00,
         4.1151e+00, -3.3734e+00,  8.5791e+00, -1.6399e-01, -4.7770e+00,
        -6.6632e+00,  1.2612e+00,  3.4832e-01,  2.7300e-01, -6.9180e+00,
        -3.3809e+00, -1.0034e+00, -2.4520e+00, -6.5167e+00, -6.3018e-02,
         2.3951e+00, -1.9835e+00,  7.9214e-01, -4.8595e+00, -1.2132e-01,
        -1.2450e+00,  3.2937e+00, -2.7142e+00,  1.0350e+00,  5.7476e-01,
         3.3565e+00, -2.1900e-01,  2.7152e+00, -3.0205e+00,  1.4406e+00,
         3.8987e+00,  7.3126e-01, -1.8026e+00,  3.6793e+00,  8.9214e+00,
         6.1839e+00, -6.8812e-01,  4.2864e+00, -1.7509e+00,  2.7934e+00,
        -5.8923e+00, -5.8426e+00, -1.1165e+00,  3.6966e+00,  8.4448e-01,
        -3.8352e+00, -4.7585e+00,  1.9213e-01,  9.1397e-01,  5.3407e+00,
         6.0364e+00,  1.1485e+00,  2.6401e+00,  7.1842e+00, -7.7628e+00,
         9.5177e-01, -5.6617e+00,  1.9012e+00, -9.2